# My first chatbot

It's just a silly chatbot, just to test regexes and the mechanism of a chatbot.

First I create a state machine to handle the chatbot states and their transition.

In [1]:
from enum import Enum
from enum import auto

# Define an enum for states
class State(Enum):
    LAMBDA = auto()
    USER_GREETED = auto()
    USER_INTRODUCED = auto()
    USER_FINISHED = auto()

In [2]:
# List of triggers which may change the machine's state
class Trigger(Enum):
    SAID_GREETING = auto()
    INTRODUCED = auto()
    SAID_GOODBYE = auto()
    INVALID_INPUT = auto()

In [3]:
# Actions are functions that are called while transiting
# from a state to another
from abc import ABC, abstractmethod
class Action(ABC):
    @abstractmethod
    def exec(self, current_state, next_state, trigger, data):
        return

class InvalidInputAction(Action):
    def exec(self, current_state, next_state, trigger, data):
        print('متوجه نشدم. شاید بهتر باشه بهم سلام کنی یا خودتو معرفی کنی!')

class SayHelloAction(Action):
    def exec(self, current_state, next_state, trigger, data):
        print('درود بر تو! میشه خودتو معرفی کنی؟')

class WhyNotHelloAction(Action):
    def exec(self, current_state, next_state, trigger, data):
        print('{} عزیز، خیلی خوب میشد اگه قبل از گفتن اسمت بهم سلام می‌کردی :)'.format(data['name']))

class SayGoodbyeAction(Action):
    def exec(self, current_state, next_state, trigger, data):
        if data and data['name']:
            print('روز خوبی داشته باشی {} عزیز'.format(data['name']))
        else:
            print('روز خوبی داشته باشی')
            
class SayHelloAgainAction(Action):
    def exec(self, current_state, next_state, trigger, data):
        print('بازم سلام :) چی باید صدات کنم؟')

class SayTheirNameAction(Action):
    def exec(self, current_state, next_state, trigger, data):
        if data and data['name']:
            print('سلام {}، از آشناییت خوشبختم :)'.format(data['name']))
        else:
            print('افتادیم تو سیاه‌چاله')

In [4]:
# The state machine class handles the machine transitions and
# executing the actions
class StateMachine:
    def __init__(self, initial_state):
        self.state = initial_state
        self.transitions = []
        self.data = {}

    # Add transition. If a transition with the same starting state and trigger
    # exists, this new transition is replaced
    def add_transition(self, from_state, to_state, trigger, action):
        replaced = False
        for i in range(len(self.transitions)):
            if self.transitions[i]['from_state'] == from_state and self.transitions[i]['trigger'] == trigger:
                replaced = True
                self.transitions[i] = {
                    'from_state': from_state,
                    'to_state': to_state,
                    'trigger': trigger,
                    'action': action
                }

        if replaced == False:
            self.transitions.append({
                'from_state': from_state,
                'to_state': to_state,
                'trigger': trigger,
                'action': action
            });

    def handle_trigger(self, trigger):
        for transition in self.transitions:
            if transition['from_state'] == self.state and transition['trigger'] == trigger:
                if transition['action'] != None:
                    transition['action'].exec(self.state, transition['to_state'], trigger, self.data)
                self.state = transition['to_state']
                break

    def set_data(self, data):
        self.data = data

In [5]:
# Instantiate the state machine and add transitions
state_machine = StateMachine(State.LAMBDA)
state_machine.add_transition(State.LAMBDA, State.LAMBDA, Trigger.INVALID_INPUT, InvalidInputAction())
state_machine.add_transition(State.LAMBDA, State.USER_GREETED, Trigger.SAID_GREETING, SayHelloAction())
state_machine.add_transition(State.LAMBDA, State.USER_INTRODUCED, Trigger.INTRODUCED, WhyNotHelloAction())
state_machine.add_transition(State.LAMBDA, State.USER_FINISHED, Trigger.SAID_GOODBYE, SayGoodbyeAction())

state_machine.add_transition(State.USER_GREETED, State.USER_GREETED, Trigger.INVALID_INPUT, InvalidInputAction())
state_machine.add_transition(State.USER_GREETED, State.USER_GREETED, Trigger.SAID_GREETING, SayHelloAgainAction())
state_machine.add_transition(State.USER_GREETED, State.USER_INTRODUCED, Trigger.INTRODUCED, SayTheirNameAction())
state_machine.add_transition(State.USER_GREETED, State.USER_FINISHED, Trigger.SAID_GOODBYE, SayGoodbyeAction())

state_machine.add_transition(State.USER_INTRODUCED, State.USER_INTRODUCED, Trigger.INVALID_INPUT, InvalidInputAction())
state_machine.add_transition(State.USER_INTRODUCED, State.USER_INTRODUCED, Trigger.SAID_GREETING, SayHelloAgainAction())
state_machine.add_transition(State.USER_INTRODUCED, State.USER_INTRODUCED, Trigger.INTRODUCED, SayTheirNameAction())
state_machine.add_transition(State.USER_INTRODUCED, State.USER_FINISHED, Trigger.SAID_GOODBYE, SayGoodbyeAction())

Now, we need to read user input in a loop, and check for triggers.

In [6]:
import re

greeting_re = re.compile(
    r'سلام|درود|چطوری.*'
)
introduce_re = re.compile(
    r'(اسم\s+من|من)\s+(.+)(\s+است|ه|م)'
)
goodbye_re = re.compile(
    r'خدافظ|خداحافظ|روز خوش|بدرود|فعلا'
)

In [7]:
def handle_input(str):
    if greeting_re.match(str):
        state_machine.handle_trigger(Trigger.SAID_GREETING)
        return
    if goodbye_re.match(str):
        state_machine.handle_trigger(Trigger.SAID_GOODBYE)
        return
    name = introduce_re.match(str)
    if name:
        state_machine.set_data({ 'name': name.groups()[-2] })
        state_machine.handle_trigger(Trigger.INTRODUCED)
    else:
        state_machine.handle_trigger(Trigger.INVALID_INPUT)

In [8]:
while state_machine.state != State.USER_FINISHED:
    user_input = input('> ')
    handle_input(user_input)

>  درود


درود بر تو! میشه خودتو معرفی کنی؟


>  اسم من علی هست


سلام علی ، از آشناییت خوشبختم :)


>  اسم من علیه


سلام علی، از آشناییت خوشبختم :)


>  بدرود


روز خوبی داشته باشی علی عزیز
